In [1]:
import pandas as pd
import numpy as np

from crossval import cvpreds_df_enhancer_folds, cvpreds_df_chunk_folds, ChunkCV,\
                      pick_train_chunks, df_cv_split
from models import DeepSeaSNP, Conservation, SNPContext, MixedModel
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score
from sklearn.preprocessing.label import LabelBinarizer

from models import DSDataKerasModel
from utils import get_seqs_and_inds
from cagi5_utils import get_breakpoint_df

/home/arh96/genseqenv/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv('data/cagi5_df.csv')
refs, alts, inds = get_seqs_and_inds(df)
df['ref_sequence'] = refs
df['alt_sequence'] = alts
df['snp_index'] = inds

In [3]:
nfolds = 5
breakpoint_df = get_breakpoint_df(df)
fold_dict = df_cv_split(breakpoint_df, nfolds)

In [4]:
cv_chunk = ChunkCV(df, DSDataKerasModel, model_kwargs={'classifier': 'xgb',
                                                       'feattypes':['diff'],
                                                       'experiment_name': 'gen2dq100_embed_500_deepseadata'},
                   nf=nfolds, fold_dict=fold_dict)
kfold_preds = cv_chunk.get_fold_preds(1)
# binarizer = LabelBinarizer()

# ybin = binarizer.fit_transform(cvdf_chunk['class'])
# print(binarizer.classes_)
# print(roc_auc_score(ybin, binarizer.transform(cvdf_chunk['cv_prediction']), average=None))

/home/arh96/genseqenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla M2090 (CNMeM is enabled with initial size: 20.0% of memory, cuDNN not available)
/home/arh96/genseqenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an a

In [5]:
model = DSDataKerasModel(experiment_name='gen2dq100_embed_500_deepseadata')

In [6]:
model.get_trained_model()

In [10]:
model.model_class.layer_activations(2, np.zeros((10,1000,4)))

array([[[0.        , 0.        , 0.        , ..., 0.01442761,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.01442761,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.01442761,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.01442761,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.01442761,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.01442761,
         0.        , 0.        ]],

       [[0.        , 0.        , 0.        , ..., 0.01442761,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.01442761,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.01442761,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.01442761,
         0.        , 0.        ],
        [0. 

In [4]:
# cv_chunk.model.model_class.model.layers[5]
cv_chunk = ChunkCV(df, DSDataKerasModel, model_kwargs={'classifier': 'xgb',
                                                       'feattypes':['diff'],
                                                       'experiment_name': 'gen2dq100_embed_500_deepseadata', 
                                                       'layers': [-1]},
                   nf=nfolds, fold_dict=fold_dict)
bkfold_preds = cv_chunk.get_fold_preds(1)

/home/arh96/genseqenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla M2090 (CNMeM is enabled with initial size: 20.0% of memory, cuDNN not available)


MissingInputError: Input 0 of the graph (indices start from 0), used to compute if{}(keras_learning_phase, Elemwise{true_div,no_inplace}.0, Reshape{3}.0), was not provided and not given a value. Use the Theano flag exception_verbosity='high', for more information on this error. 
Backtrace when that variable is created:

  File "<frozen importlib._bootstrap>", line 222, in _call_with_frames_removed
  File "/home/arh96/genseqenv/lib/python3.5/site-packages/keras/backend/__init__.py", line 80, in <module>
    from .theano_backend import *
  File "<frozen importlib._bootstrap>", line 969, in _find_and_load
  File "<frozen importlib._bootstrap>", line 958, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 673, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 662, in exec_module
  File "<frozen importlib._bootstrap>", line 222, in _call_with_frames_removed
  File "/home/arh96/genseqenv/lib/python3.5/site-packages/keras/backend/theano_backend.py", 

In [ ]:
cv_chunk = ChunkCV(df, DSDataKerasModel, model_kwargs={'classifier': 'lr',
                                                       'feattypes':['diff'],
                                                       'experiment_name': 'gen2dq100_embed_500_deepseadata'},
                   nf=nfolds, fold_dict=fold_dict)
lrkfold_preds = cv_chunk.get_fold_preds(1)
# binarizer = LabelBinarizer()

# ybin = binarizer.fit_transform(cvdf_chunk['class'])
# print(binarizer.classes_)
# print(roc_auc_score(ybin, binarizer.transform(cvdf_chunk['cv_prediction']), average=None))

In [ ]:
cv_chunk = ChunkCV(df, DeepSeaSNP, model_kwargs={'classifier': 'xgb',
                                                 'feattypes':['diff'],
                                                 },
                   nf=nfolds, fold_dict=fold_dict)
fold_preds = cv_chunk.get_fold_preds(1)

In [ ]:
cv_chunk = ChunkCV(df, DeepSeaSNP, model_kwargs={'classifier': 'lr',
                                                 'feattypes':['diff'],
                                                 },
                   nf=nfolds, fold_dict=fold_dict)
lrfold_preds = cv_chunk.get_fold_preds(1)

## Todo: make sense of the fact that lr classifier is so much worse with my dq model than with deepsea --- doesn't seem reasonable

In [6]:
fold_preds

array([0, 1, 1, ..., 0, 0, 0])

In [11]:
y = cv_chunk.breakpoint_df[~cv_chunk.breakpoint_df['is_train']]['class']

In [20]:
binarizer = LabelBinarizer()
ybin = binarizer.fit_transform(y)
print(binarizer.classes_)
print(roc_auc_score(ybin, binarizer.transform(fold_preds), average=None))
# 0.655, 0.643, 0.599

[-1  0  1]
[0.65530591 0.64384128 0.59986708]


In [13]:
print(roc_auc_score(ybin, binarizer.transform(kfold_preds), average=None))

[0.65862119 0.64893281 0.59850179]


In [16]:
print(roc_auc_score(ybin, binarizer.transform(lrkfold_preds), average=None))

[0.53977605 0.50403111 0.61289143]


In [22]:
print(roc_auc_score(ybin, binarizer.transform(lrfold_preds), average=None))
# 0.656, 0.582, 0.648

[0.6563671  0.58241205 0.64830653]


In [24]:
pd.get_dummies(breakpoint_df['regulatory_element']).values

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]], dtype=uint8)